# pypmed

In [82]:
import requests
from xml.etree import ElementTree as ET
from datetime import datetime
current_year = datetime.now().year

import requests
import time
from requests.exceptions import RequestException
from typing import Dict, List






                            



In [83]:
search_criteria = {'author_first_name': 'rachel', 
                   'author_last_name': 'gottschalk', 
                   'institution':  'university of pittsburgh at pittsburgh',
                   'publication_year': 2022
                   }
response = query_pubmed(search_criteria=search_criteria)

a
b


In [84]:
response


[{'pmid': '35063833',
  'journal_issn': '1879-0372',
  'journal_title': 'Current opinion in immunology',
  'article_title': 'Mechanisms encoding STAT functional diversity for context-specific inflammatory responses.',
  'publication_year': '2022',
  'author_first_name': 'rachel',
  'author_last_name': 'gottschalk',
  'institution': 'university of pittsburgh at pittsburgh'}]